In [123]:
# @todo -> do we get the trips or the graph as input?
# define our trips as list of departure nodes and their connections
trips = {
    # departure node, then as list the array of connections with the planned departure time, the neighbor station, the planned arrival time, the trip identifier and the tuple of actual departure and arrival times
    'Zurich': [(5, 'Olten', 15, 'IC6', [(5, 15), (6, 16), (10, 20)])],
    'Olten': [(20, 'Bern', 30, 'IC6', [(20, 30), (20, 32), (25, 35)])],
    'Bern': [(35, 'Brig', 45, 'IC6', [(35, 45), (37, 47), (40, 50)])],
    'Brig': [(55, 'Milan', 65, 'IC6', [(55, 65), (57, 67), (60, 70)])]
}

# use a dictionary structure for each trip, so we can extend it with additional information
station_trips_dict = {
    'Zurich': [
        { "from": "Zurich", "planned_departure": 5, "to": "Olten", "planned_arrival": 15, "trip_id": "IC6", "actual_times": [(5, 15), (6, 15), (10, 20)] }
    ],
    'Olten': [
        { "from": "Olten", "planned_departure": 20, "to": "Bern", "planned_arrival": 30, "trip_id": "IC6", "actual_times": [(20, 30), (20, 32), (25, 35)] }
    ],
    'Bern': [
        { "from": "Bern", "planned_departure": 35, "to": "Brig", "planned_arrival": 45, "trip_id": "IC6", "actual_times": [(35, 45), (37, 47), (40, 50)] }
    ],
    'Brig': [
        { "from": "Brig", "planned_departure": 55, "to": "Milan", "planned_arrival": 65, "trip_id": "IC6", "actual_times": [(55, 65), (57, 67), (60, 70)] }
    ]
}

In [124]:
from typing import List


def compute_probability(time: int, time_distribution: List[int]) -> float:
    """
    Compute the probability of a train departing or arriving at a given time
    given a list of departure times or arrival times
    """
    # count the number of trains departing/arriving at the given time
    count = time_distribution.count(time)
    # count the total number of trains
    total = len(time_distribution)
    # compute the probability
    probability = count / total
    return probability

def compute_probability_to_arrive_at_or_before(time_limit: int, arrival_times: List[int] | None, arrival_probabilities=None) -> float:
    """
    Compute the probability of arriving at or before a given time
    given a list of arrival times
    """
    # if we have arrival probabilities given, we can use them directly (e.g., probability to arrive at time t' given that we made the connection)
    if arrival_probabilities is not None:
        # use arrival times from the probabilities
        arrival_times = [arrival_time[0] for arrival_time in arrival_probabilities]
    # get first (minimum) arrival time
    min_arrival_time = min(arrival_times) # note: should be the first, we assume we get a sorted list
    # get last (maximum) arrival time -> either the latest arrival time or the time limit if that is earlier
    max_arrival_time = min(max(arrival_times), time_limit)

    if arrival_probabilities is not None:
        # if we have arrival probabilities given, we can use them directly (e.g., probability to arrive at time t' given that we made the connection)
        probabilities = [
            arrival_time[1]
            for arrival_time in arrival_probabilities
            if arrival_time[0] <= max_arrival_time
        ]
        # sum the probabilities
        probability = sum(probabilities)
        return probability

    # computation of probabilities, if we have only the arrival times

    # for each arrival time, compute the probability of arriving at that time (which is the probability of arriving at or before the time limit)
    probabilities = [
        compute_probability(time, arrival_times)
        for time in range(min_arrival_time, max_arrival_time + 1)
    ]
    # sum the probabilities
    probability = sum(probabilities)
    return probability

In [125]:
# Add the departure time probabilities to the data
def add_departure_probabilities(station_trips: dict):
    """
    Add the departure time probabilities to the data
    """
    for station_trips in station_trips.values():
        # loop through the trips in the station
        for trip in station_trips:
            # get the actual departure times
            actual_departure_times = [actual_time[0] for actual_time in trip["actual_times"]]
            # for each actual departure time, compute the probability of departing at that time
            departure_probabilities = [
                (time, compute_probability(time, actual_departure_times))
                for time in actual_departure_times
            ]
            # add the departure probabilities to the trip
            trip["departure_probabilities"] = departure_probabilities

add_departure_probabilities(station_trips_dict)

station_trips_dict

{'Zurich': [{'from': 'Zurich',
   'planned_departure': 5,
   'to': 'Olten',
   'planned_arrival': 15,
   'trip_id': 'IC6',
   'actual_times': [(5, 15), (6, 15), (10, 20)],
   'departure_probabilities': [(5, 0.3333333333333333),
    (6, 0.3333333333333333),
    (10, 0.3333333333333333)]}],
 'Olten': [{'from': 'Olten',
   'planned_departure': 20,
   'to': 'Bern',
   'planned_arrival': 30,
   'trip_id': 'IC6',
   'actual_times': [(20, 30), (20, 32), (25, 35)],
   'departure_probabilities': [(20, 0.6666666666666666),
    (20, 0.6666666666666666),
    (25, 0.3333333333333333)]}],
 'Bern': [{'from': 'Bern',
   'planned_departure': 35,
   'to': 'Brig',
   'planned_arrival': 45,
   'trip_id': 'IC6',
   'actual_times': [(35, 45), (37, 47), (40, 50)],
   'departure_probabilities': [(35, 0.3333333333333333),
    (37, 0.3333333333333333),
    (40, 0.3333333333333333)]}],
 'Brig': [{'from': 'Brig',
   'planned_departure': 55,
   'to': 'Milan',
   'planned_arrival': 65,
   'trip_id': 'IC6',
   'actu

In [126]:
def compute_connection_probability(departure_time_probabilities: List[int], arrival_time_probabilities: List[float] | None, trip: dict, transfer_time=5) -> float:
    # @todo -> refactor a bit (to make it useful for the next steps, e.g. how the parameters are passed and how the data is stored)
    """
    Compute the probability of making a connection given a list of departure times and a list of arrival time probabilities
    """
    # for each departure time, compute the probability of making a connection
    probabilities = []
    # multiply probability of departing at time t with the probability of arriving at or before time (t - transfer time)
    for i in range(len(departure_time_probabilities)):
        # only consider the arrival time probabilities that are before the departure time
        # @todo -> solve this with a precomputed list of probabilities (since we need it more often)
        # @todo -> maybe use combined (multiplied) probability that a certain train departs at time t' when it arrives at or before time t
        if arrival_time_probabilities is not None:
            arrival_probability = compute_probability_to_arrive_at_or_before(departure_time_probabilities[i][0] - transfer_time, None, arrival_time_probabilities)
        else:
            # actual arrival times for the trip (only if arrival probabilities are not given) -> for the connection between first and second trip
            actual_arrival_times = [actual_time[1] for actual_time in trip['actual_times']]
            arrival_probability = compute_probability_to_arrive_at_or_before(departure_time_probabilities[i][0] - transfer_time, actual_arrival_times)
        probabilities.append(departure_time_probabilities[i][1] * arrival_probability)
    # sum the probabilities
    probability = sum(probabilities)
    return probability

# we assume that we already have unique lists of tuples representing the departure and arrival times and their probabilities
# sort the lists by time
departure_probabilities = list(set(station_trips_dict['Olten'][0]['departure_probabilities']))
print("departure probabilities:", departure_probabilities)
departure_probabilities.sort(key=lambda x: x[0])
arrival_probabilities = []
#arrival_probabilities.sort(key=lambda x: x[0])
made_connection_1_2 = compute_connection_probability(departure_probabilities, None, station_trips_dict['Zurich'][0])
print("probability of making connection from Olten to Bern:", made_connection_1_2)

departure probabilities: [(20, 0.6666666666666666), (25, 0.3333333333333333)]
probability of making connection from Olten to Bern: 0.7777777777777777


In [127]:
# Now compute probability that trip 2 arrives at the third station given that we make the connection between the first and second trip (=between first and second station)

t = 30
departure_probabilities = list(set(station_trips_dict['Olten'][0]['departure_probabilities']))
departure_probabilities.sort(key=lambda x: x[0])
arrival_probabilities_second_stop = []
arrival_times_first_stop = [actual_time[1] for actual_time in station_trips_dict['Zurich'][0]['actual_times']]

arrival_departure_tuples_second_trip = station_trips_dict['Olten'][0]['actual_times']

def compute_probability_to_arrive_at_t_given_made_connection(time: int, departure_probabilities: List, arrival_probabilities: List | None, arrival_times_first_stop: List[int], probability_connection_made: float, arrival_departure_tuples_second_station: List, transfer_time=5) -> float:
    """ Compute the probability of arriving at time t given that we made the connection."""
    probabilities = []
    # loop through all possible departure times
    for t_dep in departure_probabilities:
        # probability of arriving at time t given the departure at t'
        # we need to count how often an arrival-departure pair occurs (divided by total number of arrival-departure pairs) and then divide by the probability that we depart at t'
        occurence_arrival_departure_pair = 0
        for pair in arrival_departure_tuples_second_station:
            if t_dep[0] == pair[0] and time == pair[1]:
                occurence_arrival_departure_pair += 1
        probability_arrival_t_departure_t_prime = occurence_arrival_departure_pair / len(arrival_departure_tuples_second_station)
        probability_departure = t_dep[1]
        probability_arrival_given_departure = probability_arrival_t_departure_t_prime / probability_departure
        # if we have arrival probabilities given, we can use them directly
        if arrival_probabilities is not None:
            probability_arrival_before_t_prime = compute_probability_to_arrive_at_or_before(t_dep[0] - transfer_time, None, arrival_probabilities)
        else:
            probability_arrival_before_t_prime = compute_probability_to_arrive_at_or_before(t_dep[0] - transfer_time, arrival_times_first_stop)

        probability = (probability_departure * probability_arrival_before_t_prime * probability_arrival_given_departure) / probability_connection_made
        probabilities.append(probability)
    probability_sum = sum(probabilities)

    return probability_sum

arrival_probability_30 = compute_probability_to_arrive_at_t_given_made_connection(t, departure_probabilities, None, arrival_times_first_stop, made_connection_1_2, arrival_departure_tuples_second_trip)
t = 32
arrival_probability_32 = compute_probability_to_arrive_at_t_given_made_connection(t, departure_probabilities, None, arrival_times_first_stop, made_connection_1_2, arrival_departure_tuples_second_trip)
t = 35
arrival_probability_35 = compute_probability_to_arrive_at_t_given_made_connection(t, departure_probabilities, None, arrival_times_first_stop, made_connection_1_2, arrival_departure_tuples_second_trip)

arrival_probabilities_second_trip = [(30, arrival_probability_30), (32, arrival_probability_32), (35, arrival_probability_35)]
arrival_probabilities_second_trip

[(30, 0.28571428571428575),
 (32, 0.28571428571428575),
 (35, 0.4285714285714286)]

In [128]:
# Now compute probability of making the connection between the second and third trip
departure_probabilities = list(set(station_trips_dict['Bern'][0]['departure_probabilities']))
made_connection_2_3 = compute_connection_probability(departure_probabilities, arrival_probabilities_second_trip, station_trips_dict['Olten'][0])
made_connection_2_3

0.6190476190476191

In [129]:
# Now compute probability that trip 3 arrives at the fourth station given that we make the connection between the second and third trip (=between second and third station)

t = 45
departure_probabilities = list(set(station_trips_dict['Bern'][0]['departure_probabilities']))
departure_probabilities.sort(key=lambda x: x[0])
arrival_probabilities_third_stop = []
arrival_times_second_stop = [actual_time[1] for actual_time in station_trips_dict['Olten'][0]['actual_times']]
arrival_departure_tuples_third_station = station_trips_dict['Bern'][0]['actual_times']

arrival_probability_45 = compute_probability_to_arrive_at_t_given_made_connection(t, departure_probabilities, arrival_probabilities_second_trip, arrival_times_second_stop, made_connection_2_3, arrival_departure_tuples_third_station)
t = 47
arrival_probability_47 = compute_probability_to_arrive_at_t_given_made_connection(t, departure_probabilities, arrival_probabilities_second_trip, arrival_times_second_stop, made_connection_2_3, arrival_departure_tuples_third_station)
t = 50
arrival_probability_50 = compute_probability_to_arrive_at_t_given_made_connection(t, departure_probabilities, arrival_probabilities_second_trip, arrival_times_second_stop, made_connection_2_3, arrival_departure_tuples_third_station)

arrival_probabilities_third_trip = [(45, arrival_probability_45), (47, arrival_probability_47), (50, arrival_probability_50)]
arrival_probabilities_third_trip

[(45, 0.15384615384615385), (47, 0.3076923076923077), (50, 0.5384615384615384)]

In [130]:
# Now, compute the reliability of the entire connection

# first we need to compute the probability of arriving at the last (third) station before the start_time + time bugdet, given that we made the previous connections
time_budget = 50
arrival_probability_last = compute_probability_to_arrive_at_or_before(time_budget, None, arrival_probabilities_third_trip)

# second, we multiply the arrival probability of the last station with the probability of making the last connection given that we made the previous connections
reliability = arrival_probability_last * made_connection_2_3
reliability

0.6190476190476191